## Realizar el Descenso de Gradiente en Python

* Se utilizarán funciones $f(\textrm{X})$ definidas.  
* Se calculará el gradiente $\nabla{f(X_k)}$ de forma numérica.  
* Se actualizará $\alpha_k$ en cada iteración.

In [47]:
import numpy as np
from time import perf_counter

#### Definimos las funciones de prueba

In [2]:
def Esfera(x):
    """
    f(x) = \\sum x_i^2
    """
    
    return np.sum(x**2)


def Rosenbrock(x):
    terminos = [
        100 * (x[i + 1] - x[i] ** 2) ** 2 + (1 - x[i]) ** 2
        for i in range(len(x) - 1)
    ]
    terminos = np.array(terminos)
    return np.sum(terminos)


def Beale(x):
    assert len(x) == 2, f"La función Beale es de dos variables, se introdujeron {len(x)}."
    (x1, x2) = x
    return (
        (1.5 - x1 + x1 * x2) ** 2
        + (2.25 - x1 + x1 * x2**2) ** 2
        + (2.625 - x1 + x1 * x2**3) ** 2
    )

#### Cálculo numérico del gradiente

In [3]:
def grad_f(x, f, h):
    """
    Input:
        f: Función.
        x: Punto a evaluar.
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        grad: Valor del gradiente.
    """
    
    # Inicializa el gradiente
    grad = np.zeros(len(x))

    # Itera sobre cada componente
    for i in range(len(x)):
        # Copia de x
        x_i = np.copy(x)

        # Se suma el espaciamiento solo en la i-esima componente
        x_i[i] = x_i[i] + h
        
        # Se calcula la i-esima componente del gradiente
        grad[i] = (f(x_i) - f(x)) / h
    return grad

#### Actualizar alfa en cada iteración

In [57]:
def actualiza_alfa(f, gradiente, x, p, alfa, h, rho = 0.5, c_1 = 10e-4, max_iter = 50):
    for i in range(max_iter):
        if f(x+p*alfa) <= f(x) + c_1 * alfa * np.dot(p, gradiente):
            return alfa
        alfa = alfa*rho
                
    print(f"No se cumplió la condición con {max_iter} iteraciones.")
    return alfa

##### Descenso del gradiente con actualización de $\alpha$

In [58]:
def descenso_gradiente2(f, x, alfa, max_iter, epsilon, h):
    """
    Input:
        f: Función objetivo.
        x: Punto inicial x_0.
        alfa: Learning rate.
        max_iter: Número máximo de iteraciones.
        epsilon: Criterio de convergencia.
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        x like: Punto solución aproximada.
    """
    
    
    # Inicializacion
    x_k = np.copy(x)
    convergencia = False

    for i in range(max_iter):
        # Actualiza la solucion
        gradiente = grad_f(x_k, f, h)
        p_k = -gradiente
        alfa = actualiza_alfa(f, gradiente, x_k, p_k, alfa, h)
        x_k = x_k + alfa * p_k 
        
        # Evalua la convergencia
        convergencia = max(abs(p_k)) < epsilon
        if convergencia:
            print(f"La función {f.__name__} converge en la iteracion: {i}")
            break

    if not convergencia:
        print(f"No se cumplio la convergencia en {max_iter} iteraciones.")

    return x_k

#### Descenso del gradiente sin actualizar $\alpha$

In [59]:
def descenso_gradiente(f, x, alfa, max_iter, epsilon, h):
    """
    Input:
        f: Función objetivo.
        x: Punto inicial x_0.
        alfa: Learning rate.
        max_iter: Número máximo de iteraciones.
        epsilon: Criterio de convergencia.
        h: Espaciamiento para el cálculo del gradiente.
        
    Output:
        x like: Punto solución aproximada.
    """
    
    
    # Inicializacion
    x_k = np.copy(x)
    convergencia = False

    for i in range(max_iter):
        # Actualiza la solucion
        p_k = -grad_f(x_k, f, h)
        x_k = x_k + alfa * p_k 
        
        # Evalua la convergencia
        convergencia = max(abs(p_k)) < epsilon
        if convergencia:
            print(f"La función {f.__name__} converge en la iteracion: {i}")
            break

    if not convergencia:
        print(f"No se cumplio la convergencia en {max_iter} iteraciones.")

    return x_k

#### Probando la función Esfera.

In [60]:
n = 4
x = np.array(n*[10], dtype=float)
alfa_1 = 0.1
max_iter = 15_000
epsilon = 10e-4
h = 10e-6
alfa_2 = 1

# Comparación con resultado real
solucion = np.array(n*[0], dtype=float)

inicio = perf_counter()
x_1 = descenso_gradiente(Esfera, x, alfa_1, max_iter, epsilon, h)
fin = perf_counter()

print(f"Tiempo de ejecución sin actualizar alfa: {(fin-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(x_1 - solucion)}")


inicio = perf_counter()
x_2 = descenso_gradiente2(Esfera, x, alfa_2, max_iter, epsilon, h)
fin = perf_counter()

print(f"Tiempo de ejecución actualizando alfa: {(fin-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(x_2 - solucion)}")

La función Esfera converge en la iteracion: 45
Tiempo de ejecución sin actualizar alfa: 28.69300 ms
Magnitud del vector error: 0.0006868986359257107
La función Esfera converge en la iteracion: 1
Tiempo de ejecución actualizando alfa: 10.45500 ms
Magnitud del vector error: 9.999421308748424e-06


#### Probando la función Rosenbrock.

In [61]:
n = 4
x = np.array(4*[2], dtype=float)
alfa_1 = 0.001
max_iter = 15_000
epsilon = 10e-4
h = 10e-6
alfa_2 = 1

# Comparación con resultado real
solucion = np.array(n*[1], dtype=float)

inicio = perf_counter()
x_1 = descenso_gradiente(Rosenbrock, x, alfa_1, max_iter, epsilon, h)
fin = perf_counter()

print(f"Tiempo de ejecución sin actualizar alfa: {(fin-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(x_1 - solucion)}")


inicio = perf_counter()
x_2 = descenso_gradiente2(Rosenbrock, x, alfa_2, max_iter, epsilon, h)
fin = perf_counter()

print(f"Tiempo de ejecución actualizando alfa: {(fin-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(x_2 - solucion)}")

La función Rosenbrock converge en la iteracion: 12143
Tiempo de ejecución sin actualizar alfa: 7994.65500 ms
Magnitud del vector error: 0.0068657639510175585
La función Rosenbrock converge en la iteracion: 11462
Tiempo de ejecución actualizando alfa: 9397.18700 ms
Magnitud del vector error: 0.011487211883003577


#### Probando la función Beale

In [65]:
x = np.array([2, 2], dtype=float)
alfa_1 = 0.002
max_iter = 15_000
epsilon = 10e-4
h = 10e-6
alfa_2 = 1

# Comparación con resultado real
solucion = np.array([3, 0.5], dtype=float)

inicio = perf_counter()
x_1 = descenso_gradiente(Beale, x, alfa_1, max_iter, epsilon, h)
fin = perf_counter()

print(f"Tiempo de ejecución sin actualizar alfa: {(fin-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(x_1 - solucion)}")


inicio = perf_counter()
x_2 = descenso_gradiente2(Beale, x, alfa_2, max_iter, epsilon, h)
fin = perf_counter()

print(f"Tiempo de ejecución actualizando alfa: {(fin-inicio) * 10e3:.5f} ms")
print(f"Magnitud del vector error: {np.linalg.norm(x_2 - solucion)}")

La función Beale converge en la iteracion: 7724
Tiempo de ejecución sin actualizar alfa: 1278.45800 ms
Magnitud del vector error: 0.003631774904621137
La función Beale converge en la iteracion: 4056
Tiempo de ejecución actualizando alfa: 861.41100 ms
Magnitud del vector error: 0.0036269319042124444
